In [ ]:
!git clone #insert here your github repo
!mv Recommender-Systems-Challenge/* ../content/ #moving the content 

Cloning into 'Recommender-Systems-Challenge'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 1133 (delta 131), reused 109 (delta 56), pack-reused 937
Receiving objects: 100% (1133/1133), 210.43 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (669/669), done.


In [ ]:
!python run_compile_all_cython.py #compiling cython files

run_compile_all_cython: Found 11 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:610:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_d

In [ ]:
pip install optuna

In [ ]:
pip install implicit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
import optuna

In [ ]:
dataset = pd.read_csv('/content/New Challenge Data/data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [ ]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('/content/New Challenge Data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [ ]:
ICM_df = pd.read_csv('/content/New Challenge Data/data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [ ]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender


In [ ]:
#tuning using Optuna

class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(5)
 
        
 
    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        alpha = trial.suggest_uniform('alpha', 0.1, 0.9)
        beta = trial.suggest_uniform('beta', 0.1, 0.9)
        gamma = trial.suggest_uniform('gamma', 0.1, 0.9)
        phi = trial.suggest_uniform('phi', 0.1, 0.9)
        #omega = trial.suggest_uniform('omega', 0.1, 0.9)
 
 
        self.MAP_vector = np.zeros(5)
        i = 0
        j=1
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            
            URM_train_ICM_all_IALS = sps.vstack([URM_train*(1-0.5293971458364172), ICM_all.T*0.5293971458364172])
            recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
            filename_IALS = 'MF_IALS_fold_number' + str(j)
            recommender_IALS.load_model('/content/drive/MyDrive/Rec_sys_models/MF_saved_2/', file_name=filename_IALS)# loading the model from file
            
            URM_train_ICM_all_SLIM = sps.vstack([URM_train*(1-0.421), ICM_all.T*0.421])
            recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
            filename_SLIM = 'SLIM_EN_fold_number' + str(j)
            recommender_SLIM.load_model('/content/drive/MyDrive/Rec_sys_models/SLIM_saved/', file_name=filename_SLIM)
            recommender_SLIM.W_sparse.data = np.power(recommender_SLIM.W_sparse.data, 0.8546)
 
            URM_train_ICM_all_RP3 = sps.vstack([URM_train, ICM_all.T])
            recommender_RP3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
            filename_RP3Beta = 'RP3_fold_number' + str(j)
            recommender_RP3Beta.load_model('/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/', file_name=filename_RP3Beta)
            
            URM_train_ICM_all_SLIM_BPR = sps.vstack([URM_train*(1-0.5842640825119053), ICM_all.T*0.5842640825119053])
            recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM_BPR, recompile_cython=False, verbose=False)            
            filename_SLIM_BPR = 'SLIM_BPR_fold_number' + str(j)
            recommender_SLIM_BPR.load_model('/content/drive/MyDrive/Rec_sys_models/SLIM_BPR_saved_2/', file_name=filename_SLIM_BPR)
 
            URM_train_ICM_all = sps.vstack([URM_train, ICM_all.T])
            hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_RP3Beta, recommender_SLIM, recommender_SLIM_BPR], 4)
            hybrid.fit([alpha,beta,gamma,phi])
 
 
            
            result_dict, _ = evaluator_validation.evaluateRecommender(hybrid)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
            j=j+1
        
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/5
        print('printing MAP: ', MAP)
 
 
 
        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=200)
 
print(study.best_params)

[I 2021-01-13 14:39:23,733] A new study created in memory with name: no-name-3360df2b-bdf9-4131-8237-b8ad789de1b7


Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detected 73 (0.26 %) cold users.
ItemKNNScoresHybridNRecommender: URM Detected 1 (0.00 %) cold items.
number of recommenders

[I 2021-01-13 14:42:22,607] Trial 0 finished with value: 0.07670165227242652 and parameters: {'alpha': 0.48698624080845365, 'beta': 0.25522143025091176, 'gamma': 0.3060750119282436, 'phi': 0.8294328452323576}. Best is trial 0 with value: 0.07670165227242652.


printing self map vector:  [0.08271736 0.07537671 0.07683956 0.07490096 0.07367367]
printing MAP:  0.07670165227242652
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detec

[I 2021-01-13 14:44:06,095] Trial 1 finished with value: 0.07715712054771534 and parameters: {'alpha': 0.8764592006482649, 'beta': 0.6776552592055533, 'gamma': 0.5830954230604734, 'phi': 0.7402261307154703}. Best is trial 1 with value: 0.07715712054771534.


...

printing self map vector:  [0.08447778 0.07716307 0.07857532 0.07649981 0.07430565]
printing MAP:  0.07820432641149391
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detec

[I 2021-01-13 20:22:31,876] Trial 198 finished with value: 0.07820905594965817 and parameters: {'alpha': 0.13191358256003732, 'beta': 0.8285552256844937, 'gamma': 0.13172833864416508, 'phi': 0.3857981777781643}. Best is trial 94 with value: 0.07853802215721162.


printing self map vector:  [0.08443433 0.07731431 0.07855562 0.07647077 0.07427025]
printing MAP:  0.07820905594965817
Recommender_Base_Class: URM Detected 73 (0.26 %) cold users.
Recommender_Base_Class: URM Detected 1 (0.00 %) cold items.
Recommender_Base_Class: Loading model from file '/content/drive/MyDrive/Rec_sys_models/MF_saved_2/MF_IALS_fold_number1'
Recommender_Base_Class: Loading complete
SLIMElasticNetRecommender: URM Detected 73 (0.26 %) cold users.
SLIMElasticNetRecommender: URM Detected 1 (0.00 %) cold items.
SLIMElasticNetRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/SLIM_saved/SLIM_EN_fold_number1'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
RP3betaRecommender: Loading model from file '/content/drive/MyDrive/Rec_sys_models/RP3_saved_2/RP3_fold_number1'
RP3betaRecommender: Loading complete
ItemKNNScoresHybridNRecommender: URM Detec

[I 2021-01-13 20:24:14,394] Trial 199 finished with value: 0.07845621561530558 and parameters: {'alpha': 0.1573131120494617, 'beta': 0.8503818243590131, 'gamma': 0.100081571393752, 'phi': 0.36215964679743773}. Best is trial 94 with value: 0.07853802215721162.


printing self map vector:  [0.08463448 0.07759202 0.07860546 0.07689695 0.07455216]
printing MAP:  0.07845621561530558
{'alpha': 0.15124234700704609, 'beta': 0.8800220938081902, 'gamma': 0.1010413476756059, 'phi': 0.12173192879318821}
